In [ ]:
from collections import Counter

from src.tree_parser import *

def iter_trees():
    with open("../tmp/im2latex_formulas.tree", "rb") as lines:
        for elements in iter_tree_lines(lines):
            try:
                yield list(iter_elements(elements)) if elements else []
            except:
                print(b''.join(elements))
                raise


In [ ]:
with open("../tmp/im2latex_formulas.lst", "rb") as formulas:
    formulas_number = 0
    for _ in formulas:
        formulas_number += 1
formulas_number

In [ ]:
class IsOkVisitor(Visitor):
    def __init__(self):
        self.is_ok = True
        
    def visit_with_children(self, with_children):
        if with_children.header.startswith((b'\\vbox', b'\\hbox')):
            self.is_ok = False
            
def is_ok(elements):
    visitor = IsOkVisitor()
    AllVisitor(visitor).visit_list(elements)
    return visitor.is_ok

sum(1 for elements in iter_trees() if is_ok(elements))